In [1]:
import pandas as pd
from datetime import timedelta, datetime
import numpy as np
from scipy import stats

import itertools

# data visualization 
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import statsmodels.api as sm

%matplotlib inline

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

from acquire import get_all_data

In [47]:
df = get_all_data()

In [68]:
df.head()

,item_id,sale_amount,sale_date,sale_id,store_id,item_brand,item_name,item_price,item_upc12,item_upc14,...,store_zipcode,year,quarter,month,day_of_month,day_of_week,day_name,weekend_vs_weekday,sales_total,day_over_day
0,1,13.0,2013-01-01 00:00:00+00:00,1,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,...,78253,2013,1,1,1,1,Tuesday,False,10.92,NaN
1,1,11.0,2013-01-02 00:00:00+00:00,2,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,...,78253,2013,1,1,2,2,Wednesday,False,9.24,-1.68
2,1,14.0,2013-01-03 00:00:00+00:00,3,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,...,78253,2013,1,1,3,3,Thursday,False,11.76,2.52
3,1,13.0,2013-01-04 00:00:00+00:00,4,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,...,78253,2013,1,1,4,4,Friday,False,10.92,-0.84
4,1,10.0,2013-01-05 00:00:00+00:00,5,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,...,78253,2013,1,1,5,5,Saturday,True,8.40,-2.52


In [69]:
df.sort_values('sale_date').reset_index().drop(columns=('index')).head()

,item_id,sale_amount,sale_date,sale_id,store_id,item_brand,item_name,item_price,item_upc12,item_upc14,...,store_zipcode,year,quarter,month,day_of_month,day_of_week,day_name,weekend_vs_weekday,sales_total,day_over_day
0,1,13.0,2013-01-01 00:00:00+00:00,1,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,...,78253,2013,1,1,1,1,Tuesday,False,10.92,NaN
1,17,26.0,2013-01-01 00:00:00+00:00,295813,3,Ducal,Ducal Refried Red Beans,1.16,88313590791,88313590791,...,78201,2013,1,1,1,1,Tuesday,False,30.16,-186.98
2,7,32.0,2013-01-01 00:00:00+00:00,125995,10,Twinings Of London,Twinings Of London Classics Lady Grey Tea - 20 Ct,9.64,70177154004,70177154004,...,78231,2013,1,1,1,1,Tuesday,False,308.48,-114.25
3,18,45.0,2013-01-01 00:00:00+00:00,314073,3,Scotch,Scotch Removable Clear Mounting Squares - 35 Ct,4.39,21200725340,21200725340,...,78201,2013,1,1,1,1,Tuesday,False,197.55,155.79
4,19,34.0,2013-01-01 00:00:00+00:00,332333,3,Careone,Careone Family Comb Set - 8 Ct,0.74,41520035646,41520035646,...,78201,2013,1,1,1,1,Tuesday,False,25.16,-440.18


- Write a function to convert a date to adatetime data type.
- Write a function to change a datetime to UTC.

In [70]:
def convert_to_datetime(df):
    '''
    Function converts a data to adatetime data type, it also sets the time hyperparameter to UTC
    '''
    df['sale_date'] = pd.to_datetime(df['sale_date'], utc=True)
    return df

In [71]:
df = convert_to_datetime(df)

- Write a function to parse a date column into 6 additional columns (while keeping the original date): year, quarter, month, day of month, day of week, weekend vs. weekday

In [72]:
def date_breakdown(df):
    '''
    Function creates 7 additional columns: year, quarter, month, day of month, day of week, weekend vs. weekday using basic date and time types
    '''
    df['year'] = df.sale_date.dt.year
    df['quarter'] = df.sale_date.dt.quarter
    df['month'] = df.sale_date.dt.month
    df['day_of_month'] = df.sale_date.dt.day
    df['day_of_week'] = df.sale_date.dt.weekday
    df['day_name'] = df.sale_date.dt.weekday_name
    df['weekend_vs_weekday'] = df['day_of_week'] > 4
    return df

In [73]:
df = date_breakdown(df)

- Add a column to your dataframe, sales_total, which is a derived from sale_amount (total items) and item_price.

In [74]:
df['sales_total'] = df['sale_amount'] * df['item_price']

- Create a new dataframe that aggregates the sales_total and sale_amount(item count) using sum and median by day of week.

In [75]:
dfagg = df.groupby('day_name')[['sales_total', 'sale_amount']].agg(['median', 'sum'])
dfagg

sales_total               sale_amount           
               median           sum      median        sum
day_name                                                  
Friday         250.43  3.868773e+07        50.0  7198021.0
Monday         186.76  2.896118e+07        38.0  5385853.0
Saturday       266.97  4.116177e+07        53.0  7655482.0
Sunday         282.47  4.359486e+07        57.0  8109705.0
Thursday       234.52  3.628727e+07        47.0  6749880.0
Tuesday        218.66  3.383954e+07        44.0  6293481.0
Wednesday      218.92  3.392832e+07        44.0  6312090.0

- Explore the pandas DataFrame.diff() function. Create a new column that is the result of the current sales - the previous days sales.

In [66]:
df['day_over_day'] = df['sales_total'].diff()

In [67]:
df.tail()

,item_id,sale_amount,sale_date,sale_id,store_id,item_brand,item_name,item_price,item_upc12,item_upc14,...,store_zipcode,year,quarter,month,day_of_month,day_of_week,day_name,weekend_vs_weekday,sales_total,day_over_day
912995,50,63.0,2017-12-27 00:00:00+00:00,912996,10,Choice,Choice Organic Teas Black Tea Classic Black - ...,5.2,47445919221,47445919221,...,78231,2017,4,12,27,2,Wednesday,False,327.6,114.4
912996,50,59.0,2017-12-28 00:00:00+00:00,912997,10,Choice,Choice Organic Teas Black Tea Classic Black - ...,5.2,47445919221,47445919221,...,78231,2017,4,12,28,3,Thursday,False,306.8,-20.8
912997,50,74.0,2017-12-29 00:00:00+00:00,912998,10,Choice,Choice Organic Teas Black Tea Classic Black - ...,5.2,47445919221,47445919221,...,78231,2017,4,12,29,4,Friday,False,384.8,78.0
912998,50,62.0,2017-12-30 00:00:00+00:00,912999,10,Choice,Choice Organic Teas Black Tea Classic Black - ...,5.2,47445919221,47445919221,...,78231,2017,4,12,30,5,Saturday,True,322.4,-62.4
912999,50,82.0,2017-12-31 00:00:00+00:00,913000,10,Choice,Choice Organic Teas Black Tea Classic Black - ...,5.2,47445919221,47445919221,...,78231,2017,4,12,31,6,Sunday,True,426.4,104.0


In [58]:
df.weekend_vs_weekday.value_counts()

False    652000
True     261000
Name: weekend_vs_weekday, dtype: int64

- Write a function to set the index to be the datetime variable.

In [153]:
def set_index(df):
    df = df.set_index('sale_date')
    return df

In [154]:
df = set_index(df)

In [155]:
df.head()

,item_id,sale_amount,sale_id,store_id,item_brand,item_name,item_price,item_upc12,item_upc14,store_address,...,store_state,store_zipcode,year,quarter,month,day_of_month,day_of_week,sales_total,weekend vs. weekday,day_over_day
sale_date,,,,,,,,,,,,,,,,,,,,,
2013-01-01 00:00:00+00:00,1,13.0,1,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,...,TX,78253,2013,1,1,1,Tuesday,10.92,False,NaN
2013-01-02 00:00:00+00:00,1,11.0,2,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,...,TX,78253,2013,1,1,2,Wednesday,9.24,False,-1.68
2013-01-03 00:00:00+00:00,1,14.0,3,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,...,TX,78253,2013,1,1,3,Thursday,11.76,False,2.52
2013-01-04 00:00:00+00:00,1,13.0,4,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,...,TX,78253,2013,1,1,4,Friday,10.92,False,-0.84
2013-01-05 00:00:00+00:00,1,10.0,5,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,...,TX,78253,2013,1,1,5,Saturday,8.40,False,-2.52
